In [1]:
import math

class TermStructure:
    def __init__(self):
        pass

    # Method to calculate yield from discount factor
    def yield_from_discount_factor(self, d_t, t):
        return - math.log(d_t) / t

    # Method to calculate discount factor from yield
    def discount_factor_from_yield(self, r, t):
        return math.exp(-r * t)

    # Method to calculate forward rate from discount factors
    def forward_rate_from_discount_factors(self, d_t1, d_t2, time):
        return (math.log(d_t1 / d_t2)) / time

    # Method to calculate forward rate from yields
    def forward_rate_from_yields(self, r_t1, r_t2, t1, t2):
        return (r_t2 * t2 - r_t1 * t1) / (t2 - t1)


class TermStructureClass:
    def r(self, t):
        """
        Return the yield at time t.
        This method needs to be implemented in a derived class.
        """
        raise NotImplementedError("Subclasses should implement this!")

    def d(self, t):
        """
        Return the discount factor at time t.
        """
        return self.discount_factor_from_yield(self.r(t), t)

    def f(self, t1, t2):
        """
        Return the forward rate between time t1 and t2.
        """
        d1 = self.d(t1)
        d2 = self.d(t2)
        return self.forward_rate_from_discount_factors(d1, d2, t2 - t1)


class TermStructureClassInterpolated(TermStructureClass, TermStructure):
    def __init__(self, times=None, yields=None, flat_rate=None):
        """
        Initialize with optional times and yields lists, and an optional flat rate.
        If a flat rate is provided, it overrides the times and yields interpolation.
        """
        self.times = times if times is not None else []
        self.yields = yields if yields is not None else []
        self.flat_rate = flat_rate  # Optional flat rate for the term structure

    def clear(self):
        """
        Clear the times and yields lists.
        """
        self.times.clear()
        self.yields.clear()

    def set_interpolated_observations(self, in_times, in_yields):
        """
        Set interpolated observation times and yields.
        """
        if len(in_times) != len(in_yields):
            raise ValueError("times and yields must have the same length")
        self.times = in_times
        self.yields = in_yields

    def r(self, T):
        """
        Return the yield at time T.
        If a flat rate is set, return it; otherwise, use linear interpolation.
        """
        if self.flat_rate is not None:
            return self.flat_rate  # Use flat rate if provided
        return self.linear_interpolation(T, self.times, self.yields)

    def linear_interpolation(self, T, times, yields):
        """
        Perform linear interpolation to find the yield at time T.
        """
        for i in range(len(times) - 1):
            if times[i] <= T <= times[i + 1]:
                # Linear interpolation formula
                return yields[i] + (yields[i + 1] - yields[i]) * (T - times[i]) / (times[i + 1] - times[i])
        raise ValueError(f"T={T} is out of bounds of the provided times")

In [2]:
# Use Example 
times = [0.1,0.5,1,5,10]
yields = [0.1,0.2,0.3,0.4,0.5]
term_structure = TermStructureClassInterpolated(times, yields)

# Let's find now some rates at the following times : 0.1, 0.5, 1
dict_rates={'spot_rate_for_0.1':term_structure.r(0.1),
           'discount_factor_for_0.5':term_structure.d(0.5),
           'forward_rate_for_0.5_and_1':term_structure.f(0.5,1)}
dict_rates

{'spot_rate_for_0.1': 0.1,
 'discount_factor_for_0.5': 0.9048374180359595,
 'forward_rate_for_0.5_and_1': 0.3999999999999997}